## Generate Notes using Random Note

In [5]:
import numpy as np
import random
import json
import pretty_midi
import tensorflow as tf
from keras import backend as K
from tqdm import tqdm_notebook
from numpy.random import choice

In [6]:
def piano_roll_to_pretty_midi(piano_roll, fs=100, program=0):
    '''Convert a Piano Roll array into a PrettyMidi object
     with a single instrument.
    Parameters
    ----------
    piano_roll : np.ndarray, shape=(128,frames), dtype=int
        Piano roll of one instrument
    fs : int
        Sampling frequency of the columns, i.e. each column is spaced apart
        by ``1./fs`` seconds.
    program : int
        The program number of the instrument.
    Returns
    -------
    midi_object : pretty_midi.PrettyMIDI
        A pretty_midi.PrettyMIDI class instance describing
        the piano roll.
    '''
    notes, frames = piano_roll.shape
    pm = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=program)

    # pad 1 column of zeros so we can acknowledge inital and ending events
    piano_roll = np.pad(piano_roll, [(0, 0), (1, 1)], 'constant')

    # use changes in velocities to find note on / note off events
    velocity_changes = np.nonzero(np.diff(piano_roll).T)

    # keep track on velocities and note on times
    prev_velocities = np.zeros(notes, dtype=int)
    note_on_time = np.zeros(notes)

    for time, note in zip(*velocity_changes):
        # use time + 1 because of padding above
        velocity = piano_roll[note, time + 1]
        time = time / fs
        if velocity > 0:
            if prev_velocities[note] == 0:
                note_on_time[note] = time
                prev_velocities[note] = velocity
        else:
            pm_note = pretty_midi.Note(
                velocity=prev_velocities[note],
                pitch=note,
                start=note_on_time[note],
                end=time)
            instrument.notes.append(pm_note)
            prev_velocities[note] = 0
    pm.instruments.append(instrument)
    return pm

In [33]:
def generate_from_random(unique_notes, seq_len=50):
    generate = np.random.randint(0,unique_notes,seq_len).tolist()
    return generate

In [51]:
def generate_notes(generate, model, unique_notes, max_generated=1000, seq_len=50):
    for i in tqdm_notebook(range(max_generated), desc='genrt'):
        test_input = np.array([generate])[:,i:i+seq_len]
        predicted_note = model.predict(test_input)
        print(len(predicted_note[0]))
        random_note_pred = choice(unique_notes, 1, replace=False, p=predicted_note[0])
        generate.append(random_note_pred[0])
    return generate

In [43]:
def write_midi_file_from_generated(generate, dict_index,midi_file_name = "result.mid", start_index=49, fs=8, max_generated=1000):
    note_string = []
    for note, ind in dict_index.items():
        for ind_note in generate:
            if ind_note == ind:
                note_string.append(note)
    array_piano_roll = np.zeros((128,max_generated+1), dtype=np.int16)
    for index, note in enumerate(note_string[start_index:]):
        if note == 'e':
            pass
        else:
            splitted_note = note.split(',')
            for j in splitted_note:
                array_piano_roll[int(j),index] = 1
    generate_to_midi = piano_roll_to_pretty_midi(array_piano_roll, fs=fs)
    print("Tempo {}".format(generate_to_midi.estimate_tempo()))
    for note in generate_to_midi.instruments[0].notes:
        note.velocity = 100
    generate_to_midi.write(midi_file_name)

In [45]:
with open('data/Chopin_index.json', 'r') as read_file:
    dict_index = json.load(read_file)

In [56]:
model = tf.keras.models.load_model('chopin_model.h5')

In [57]:
unique_notes = len(dict_index)

In [58]:
unique_notes

112090

In [59]:
max_generate = 200
seq_len=50
generate = generate_from_random(unique_notes, seq_len)
generate = generate_notes(generate, model, unique_notes, max_generate, seq_len)

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


112091


ValueError: 'a' and 'p' must have same size

In [44]:
write_midi_file_from_generated(generate,dict_index, "brahms_1.mid", start_index=seq_len-1, fs=7, max_generated = max_generate)

Tempo 209.99999999999977
